In [5]:
import pandas as pd
import os

SAFECAST_DATA_PATH = os.path.join('results', 'fukushima_filtered.csv')
df = pd.read_csv(SAFECAST_DATA_PATH)

In [6]:
df.head()

,captured time,latitude,longitude,value
0,2018-10-21 01:00:22,37.796306,140.514413,19.0
1,2018-10-21 01:00:16,37.723330,140.476797,15.0
2,2018-10-21 01:00:16,37.787500,140.552400,18.0
3,2018-10-21 01:00:16,37.796306,140.514413,61.0
4,2018-10-21 01:00:14,37.792805,140.930957,13.0


In [7]:

test = df.head()
test

,captured time,latitude,longitude,value
0,2018-10-21 01:00:22,37.796306,140.514413,19.0
1,2018-10-21 01:00:16,37.723330,140.476797,15.0
2,2018-10-21 01:00:16,37.787500,140.552400,18.0
3,2018-10-21 01:00:16,37.796306,140.514413,61.0
4,2018-10-21 01:00:14,37.792805,140.930957,13.0


In [8]:
from gmalthgtparser import HgtParser
HGTDIR = "hgt_files"

def get_file_name(lat, lon):
    
    if lat >= 0.0:
        ns = 'N'
    else:
        ns = 'S'

    if lon >= 0.0:
        ew = 'E'
    else:
        ew = 'W'

    hgt_file = "%(ns)s%(lat)02d%(ew)s%(lon)03d.hgt" % {'lat': abs(lat), 'lon': abs(lon), 'ns': ns, 'ew': ew}
    hgt_file_path = os.path.join(HGTDIR, hgt_file)
    if os.path.isfile(hgt_file_path):
        return hgt_file_path
    else:
        return None

In [9]:
def read_elevation_from_file(hgt_file, lat, lon):
    with HgtParser(hgt_file) as parser:
        return parser.get_elevation((lat, lon))[2]

In [12]:
def get_elevation(row):
    lat, lon = row.latitude, row.longitude
    hgt_file = get_file_name(lat, lon)
    if hgt_file:
        return read_elevation_from_file(hgt_file, lat, lon)
    # Treat it as data void as in SRTM documentation
    # if file is absent
    return -32768

In [15]:
df["elevation"] = df.apply(get_elevation, axis=1)

In [16]:
df.elevation

0          72
1         141
2         107
3          72
4           8
5           8
6         141
7         107
8           7
9           7
10         10
11         10
12        218
13        218
14          5
15          5
16         11
17         11
18         80
19         80
20        227
21        419
22        419
23         69
24         69
25         12
26         12
27        462
28        462
29        233
         ... 
216422    333
216423    319
216424    329
216425    350
216426    336
216427    346
216428    379
216429    375
216430    379
216431    159
216432    159
216433     10
216434     10
216435    141
216436     69
216437    141
216438     69
216439    218
216440    218
216441     80
216442     80
216443    419
216444    419
216445     11
216446     11
216447    107
216448     12
216449    462
216450    462
216451     12
Name: elevation, Length: 216452, dtype: int64

In [18]:
path = os.path.join("results", "fukushima_filtered_elevation.csv")
df.to_csv(path, index=False)